In [1]:
# General modules
import sys
import os
import session_info
import warnings
from pyprojroot.here import here
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

from tqdm import tqdm
# Specific modules
import scanpy as sc
import anndata as ad
import scienceplots

# Setting some parameters
warnings.filterwarnings("ignore")
sys.path.insert(1, str(here('bin')))

# Import functions
from customPythonFunctions import balanced_sample

plt.style.use(['nature'])
dpi_fig_save = 300
sc.set_figure_params(dpi=100, dpi_save=dpi_fig_save, vector_friendly=True)

overwriteFigures = True
overwriteData = True

pd.options.display.max_columns=300
pd.options.display.max_rows = 1000

**Defining functions**

In [2]:
def qc_gex_projectlibrary(adata):
    import numpy as np
    import pandas as pd
    import anndata
    
    # Group the data by "project" and "library" using pandas
    grouped = adata.obs.groupby(["libraryID"])
    
    # Create an empty list to store the summary table rows
    summary_rows = []

    # Iterate over each group
    for (libraryID), group_data in grouped:
        
        # Calculate the median values within the group
        chemistry = np.unique(group_data["chemistry"])
        num_cells = group_data["libraryID"].size
        median_counts = np.median(group_data["total_counts"])
        median_features = np.median(group_data["n_genes_by_counts"])
        median_mt_pct = np.median(group_data["pct_counts_mt"])
        median_ribo_pct = np.median(group_data["pct_counts_ribo"])
        median_hb_pct = np.median(group_data["pct_counts_hb"])
        median_plt_pct = np.median(group_data["pct_counts_plt"])
    
        # Create a summary row with the group information and median values
        summary_row = {
            "libraryID": libraryID,
            "chemistry": chemistry,
            "Number of cells": int(num_cells),
            "Median UMI counts": int(median_counts),
            "Median Genes": int(median_features),
            "Median MT %": round(median_mt_pct, 2),
            "Median RB %": round(median_ribo_pct,2),
            "Median HB %": round(median_hb_pct,2),
            "Median PLT %": round(median_plt_pct,2),
        }

        # Append the summary row to the list
        summary_rows.append(summary_row)

    # Create a DataFrame from the summary rows
    summary_table = pd.DataFrame(summary_rows)
    return summary_table
    
def qc_gene_expression_per_study(adata, gene_list = None, save_name=None, fig_size = (20,4)):
    import numpy as np
    import pandas as pd
    import anndata
    import sys
    import seaborn as sns
    
    if gene_list is None:
        gene_list = adata.var_names
    
    res = pd.DataFrame(columns=gene_list, index=adata.obs['studyID'].unique())
    
    for source in adata.obs['studyID'].unique(): 
        res.loc[source] = adata[adata.obs['studyID'].isin([source]),gene_list].X.mean(0)
    
    BinRes = res.copy()
    BinRes[BinRes > 0 ] = 1.0
    
    sys.setrecursionlimit(50000)
    sns.clustermap(BinRes.astype(float), z_score=None, figsize=fig_size)

    if save_name:
        plt.savefig(here(save_name), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    plt.show()

In [3]:
print("Main directory path: {}".format(here()))

# create result directories
os.makedirs(here('00_data_processing/results/extended_obs'), exist_ok = True)
os.makedirs(here('00_data_processing/figures'), exist_ok=True)

Main directory path: /scratch_isilon/groups/singlecell/shared/projects/Inflammation-PBMCs-Atlas-R1


**Load data**

In [4]:
# Load the h5ad file
infile_path = here("00_data_processing/results/01_INFLAMMATION_core.h5ad")
adata = sc.read_h5ad(infile_path)
adata

AnnData object with n_obs × n_vars = 6016705 × 37169
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status'

In [5]:
adata.obs.head()

,studyID,libraryID,sampleID,chemistry,technology,patientID,disease,timepoint_replicate,treatmentStatus,therapyResponse,sex,age,BMI,binned_age,diseaseStatus,smokingStatus,ethnicity,institute,diseaseGroup
cellID,,,,,,,,,,,,,,,,,,,
SCGT00_L051_I57.3P_T0_AAACCCAAGGTGAGAA,SCGT00,SCGT00_L051,SCGT00_I57.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I57,RA,0.0,ongoing,NR,female,47.0,25.71,41-50,na,never-smoker,na,na,IMIDs
SCGT00_L051_I56.3P_T0_AAACCCAAGTCCGTCG,SCGT00,SCGT00_L051,SCGT00_I56.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I56,RA,0.0,ongoing,R,female,56.0,30.11,51-60,na,never-smoker,na,na,IMIDs
SCGT00_L051_I53.3P_T0_AAACCCAAGTGCACTT,SCGT00,SCGT00_L051,SCGT00_I53.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I53,RA,0.0,ongoing,R,female,69.0,25.00,61-70,na,never-smoker,na,na,IMIDs
SCGT00_L051_I52.3P_T0_AAACCCACAACTGTGT,SCGT00,SCGT00_L051,SCGT00_I52.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I52,RA,0.0,ongoing,NR,female,33.0,21.37,31-40,na,never-smoker,na,na,IMIDs
SCGT00_L051_I56.3P_T0_AAACCCACAAGAATGT,SCGT00,SCGT00_L051,SCGT00_I56.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I56,RA,0.0,ongoing,R,female,56.0,30.11,51-60,na,never-smoker,na,na,IMIDs


In [6]:
adata.var.head()

,hgnc_id,symbol,locus_group,HUGO_status
ensembl_gene_id,,,,
ENSG00000000003,HGNC:11858,TSPAN6,protein_coding,official
ENSG00000000005,HGNC:17757,TNMD,protein_coding,official
ENSG00000000419,HGNC:3005,DPM1,protein_coding,official
ENSG00000000457,HGNC:19285,SCYL3,protein_coding,official
ENSG00000000460,HGNC:25565,FIRRM,protein_coding,official


**Load metadata**

In [7]:
datasetDF = pd.read_csv(here('00_data_processing/datasets/INFLAMMATION_ATLAS_datasetsR1__6March2024_curated.csv'))
datasetDF.set_index('studyID', inplace=True)
datasetDF

,diseaseGroup,disease,dataSource,h5ad_FileName,ENSEMBLGeneID,externalValidation,chemistry,technology,cellrangerVersion,genomeVersion,expectedDoubletRate
studyID,,,,,,,,,,,
SCGT00,homeostasis;IMID,HC;RA;PsA;IBDs_CD;IBDs_UC;PS;SLE,inhouse,INFLAMMATION_DOCTIS_week0_bothChemistry_merged...,YES,NO,mixed,mixed_GenoHashed,v6.1.1_NoIntrons,GRCh38_v32/Ensembl98,0.02
SCGT01,homeostasis,HC,inhouse,INFLAMMATION_EXERCISEHC_merged.h5ad,YES,NO,3_GEX_V3,3_GEX_V3_CellPlex,v6.1.1_NoIntrons,GRCh38_v32/Ensembl98,0.02
SCGT02,homeostasis;chronic_inflammation,HC;Asthma;COPD,inhouse,INFLAMMATION_ASTHMA-COPD-HC_merged.h5ad,YES,NO,3_GEX_V3,3HT_GEX_V3_GenoHashed,v6.1.1_NoIntrons,GRCh38_v32/Ensembl98,0.01
SCGT03,solid_tumors,BRCA,inhouse,INFLAMMATION_BRCA_time0_merged.h5ad,YES,NO,5_GEX_V2,5_GEX_V2,v6.1.1_NoIntrons,GRCh38_v32/Ensembl98,0.08
SCGT04,chronic_inflammation,cirrhosis,inhouse,INFLAMMATION_CIRRHOSIS_merged.h5ad,YES,NO,3_GEX_V3,3HT_GEX_V3_CellPlex,v6.1.1_NoIntrons,GRCh38_v32/Ensembl98,0.01
SCGT05,solid_tumors,CRC,inhouse,INFLAMMATION_CRC_merged.h5ad,YES,NO,5_GEX_V2,5_GEX_V2,v6.1.2_NoIntrons,GRCh38_v32/Ensembl98,0.08
SCGT06,homeostasis;infection,HC;COVID,inhouse,INFLAMMATION_COVID-HC_merged.h5ad,YES,YES,3_GEX_V3,3HT_GEX_V3_GenoHashed,v6.1.1_NoIntrons,GRCh38_v32/Ensembl98,0.01
Reyes2020,homeostasis;acute_inflammation,HC;sepsis,public,Reyes_NatMed_2020__Sepsis-HC.h5ad,NO,NO,3_GEX_V2,3_GEX_V2_Hashed,v2.1_NoIntrons,GRCh38/Ensembl87,0
Cillo2020,homeostasis;solid_tumors,HC;HNSCC,public,Cillo_Immunity_2020__HNSCC-HC.h5ad,YES,NO,3_GEX_V2,3_GEX_V2,v2.1,GRCh38,0.08


# Quality control

## Compute QC covariates

First, we will compute the QC covariates (fraction of mitochondrial, ribosomal and hemoglobine genes) on our data.

In [8]:
# mitochondrial genes
adata.var["mt"] = adata.var["symbol"].str.startswith("MT-")
# ribosomal genes
adata.var["ribo"] = adata.var["symbol"].str.startswith(("RPS", "RPL"))
# hemoglobin genes
adata.var["hb"] = adata.var["symbol"].str.contains("^HB[^(P)]")
# platelet genes
adata.var["plt"] = adata.var["symbol"].isin(["PPBP", "PDGF", "ANG1", "LAPTM4B", "WASF3", "TPM3", "PF4", "TAC1"])

sc.pp.calculate_qc_metrics(adata, qc_vars=["mt", "ribo", "hb", "plt"], inplace=True, percent_top=[20], log1p=True)
adata

AnnData object with n_obs × n_vars = 6016705 × 37169
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'

In [9]:
pd.set_option('display.max_rows', None)
qc_gex_df = qc_gex_projectlibrary(adata)
display(qc_gex_df)

pd.reset_option('display.max_rows')

,libraryID,chemistry,Number of cells,Median UMI counts,Median Genes,Median MT %,Median RB %,Median HB %,Median PLT %
0,"(COMBAT2022_L000,)",[5_GEX_V1],13932,3276,1147,1.53,29.71,0.00,0.06
1,"(COMBAT2022_L001,)",[5_GEX_V1],14390,3141,1110,1.62,30.09,0.00,0.06
2,"(COMBAT2022_L002,)",[5_GEX_V1],15324,2966,1061,1.61,30.39,0.00,0.06
3,"(COMBAT2022_L003,)",[5_GEX_V1],15625,3057,1080,1.62,30.64,0.00,0.06
4,"(COMBAT2022_L004,)",[5_GEX_V1],15641,3080,1091,1.63,30.55,0.00,0.06
5,"(COMBAT2022_L005,)",[5_GEX_V1],15524,2797,1008,1.53,30.80,0.00,0.06
6,"(COMBAT2022_L006,)",[5_GEX_V1],14990,2966,1061,1.51,30.18,0.00,0.06
7,"(COMBAT2022_L007,)",[5_GEX_V1],13421,3437,1129,2.21,30.30,0.00,0.07
8,"(COMBAT2022_L008,)",[5_GEX_V1],14402,3467,1130,2.13,30.95,0.00,0.07
9,"(COMBAT2022_L009,)",[5_GEX_V1],15077,3396,1121,2.20,30.93,0.00,0.07


# Quality control at LIBRARY level

After exploring general library QC metrics, we decided to filter out the libraries with very low quality, considering the total number of cells (standard 10X channel is designed to target 2-10K cells) and the median number of total genes (recovering information for very few genes might become a problem in the downstream analysis steps). 

In [10]:
# QC thresholds
min_cells_by_lib = 500
min_meangenes_by_lib = 500

In [11]:
# Removing libraries with very low total number of cells

# Calculate the value counts for each 'libraryID'
library_cells = adata.obs["libraryID"].value_counts()

# Get the library names to remove
libraries_to_remove = library_cells[library_cells < min_cells_by_lib].index

if len(libraries_to_remove) > 0:
    print('Libraries removed:')
    print(libraries_to_remove.astype(str))

    # Filter the 'adata' object
    adata = adata[~adata.obs["libraryID"].isin(libraries_to_remove.astype(str))]
else:
    print('No library is removed')

Libraries removed:
Index(['Ren2021_L084', 'Ren2021_L029', 'Reyes2020_L014', 'Ren2021_L081',
       'Reyes2020_L022', 'Reyes2020_L016', 'Ren2021_L086', 'Reyes2020_L001',
       'Reyes2020_L012'],
      dtype='object', name='libraryID')


In [12]:
# Removing libraries with very low median number of total genes

# Calculate the median total counts for each 'libraryID'
library_median_counts = adata.obs.groupby("libraryID")["n_genes_by_counts"].median()

# Get the library names with median "total_counts" fewer than 500
libraries_to_remove = library_median_counts[library_median_counts < min_meangenes_by_lib].index

if len(libraries_to_remove) > 0:
    print('Libraries removed:')
    print(libraries_to_remove.astype(str))

    # Filter the 'adata' object
    adata = adata[~adata.obs["libraryID"].isin(libraries_to_remove.astype(str))]
else:
    print('No library is removed')

Libraries removed:
Index(['Perez2022_L016', 'Perez2022_L068', 'Perez2022_L069', 'Perez2022_L070',
       'Perez2022_L084', 'Perez2022_L085', 'Perez2022_L086', 'Perez2022_L087',
       'Reyes2020_L018', 'Reyes2020_L024', 'Reyes2020_L025', 'Reyes2020_L026',
       'Zhang2023_L000', 'Zhang2023_L004'],
      dtype='object', name='libraryID')


# Quality control at CELL level

We aim to remove low-quality cells from the dataset focusing on the main three QC metrics, but also two additional QC covariates interesting:

1. Library size: Number of counts or UMIs per barcode
2. Library complexity: Number of genes per barcode
3. Fraction of counts from mitochondrial genes per barcode
4. (Fraction of counts from ribosomal genes per barcode)
5. (Fraction of counts from hemoglobine genes per barcode)

Following best practices: https://www.sc-best-practices.org/preamble.html

Luecken, M. D., & Theis, F. J. (2019). Current best practices in single‐cell RNA‐seq analysis: a tutorial. Molecular systems biology, 15(6), e8746

## Doublet detection

Second, we will detect **doublets** or **multiplets** defined as 2 or more cells that are captured and sequenced under the same cellular barcode. Currently, there are multiple methods to identify doublets and remove them to avoid distorted downstream analysis.

Considering the nature of our data, this step has already been taken into consideration in some of the included datasets:
* Annotated and/or publicaly available datasets that have already been processed and analyzed might have also removed doublets.
* In the 10X libraries generated following a multiplexing approach (Genotype, HTO, or CellPlex), heterotypic doublets from different samples have already been detected and excluded. However, in these cases, homotypic doublets (from the same patient) might not have been captured.

As explained above, we depending on the chemistry approach used to generate the 10XGenomics library and their subsequent preprocessing steps, we can know it advance the expected doublet rate for each library:
* For standard 10XGenomics assays, expected doublet rate is 8% (0.08) per 10,000 cells.
* For High-Throughput 10XGenomics assays, expected doublet rate is 4% (0.04) per 10,000 cells.
* After a demultiplexing step, where cells containing samples from different patients would be discarded as doublets -but not cells from different or equal biological identity-, we considered the expected doublet rate to drop 75%, then being 2% (0.02) or 1% (0.01) per 10,000 cells.
* If library was annotated and doublets were flagged and removed, then expeceted doublet rate is 0.


For this reason, and to be fair in terms of doublet detection across datasets, we will run SCRUBLET (https://www.cell.com/cell-systems/fulltext/S2405-4712(18)30474-5?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS2405471218304745%3Fshowall%3Dtrue) following the tutorials on https://github.com/swolock/scrublet, in order to predict which cell barcodes might be multiplets, and tagged them.

In [13]:
# Assess datasetDF
datasetDF[['expectedDoubletRate']]

## at this point, not present expected_doublet_rate column, I need to regenerate with a criteria, but its difficult.

,expectedDoubletRate
studyID,
SCGT00,0.02
SCGT01,0.02
SCGT02,0.01
SCGT03,0.08
SCGT04,0.01
SCGT05,0.08
SCGT06,0.01
Reyes2020,0
Cillo2020,0.08


In [14]:
# Run SCRUBLET by libraryID
if not os.path.isfile(here('00_data_processing/results/02_scrublet_score.pkl')):
    libraries_list = np.unique(adata.obs["libraryID"])
    # Create an empty DataFrame
    # scrublet_results_df = pd.DataFrame(columns=['cellID', 'doublet_score', 'predicted_doublet']) 
    scrublet_results_df_list = []
    
    # Loop through the unique ILCIC libraries
    for library in tqdm(libraries_list): #REMOVE THE INDEX!!
    
        # print(library)
        
        studyID = library.split('_L')[0] 
       
        # Subset AnnData object based on the current library
        adata_sub = adata[adata.obs["libraryID"] == library].copy()
       
        # Parameters
        # Expected doublet ratio 0.08 for standard GEX libraries, whereas 0.04 for HT GEX libraries; and should be near 0 for filtered & cleaned datasets
        # Additionally, for multiplexed technologies, as many doublets have already been removed, will be lower.
        # We will use a specific expected doublet ratio based on the preprocess already applied on each dataset.
    
        expected_doublet_rate_string = datasetDF.loc[studyID].expectedDoubletRate
        
        if datasetDF.loc[studyID].expectedDoubletRate == 'mixed':
            studyTechnology = adata_sub.obs.technology.unique().tolist()
            assert (len(studyTechnology) == 1)
            studyTechnology = studyTechnology[0]
            
            if (studyTechnology == '3_GEX_V3') or (studyTechnology == '5_GEX_V2'):
                expected_doublet_rate = 0.08
            elif studyTechnology == '3HT_GEX_V3':
                expected_doublet_rate = 0.04
            else:
                print(f"ERRROR! expected double rate for technology {studyTechnology} not defined")
                raise
        else:
            expected_doublet_rate = float(expected_doublet_rate_string)
            
        expected_doublet_scaled_rate = (adata_sub.n_obs * expected_doublet_rate) / 10000 # scale the ratio considering the actual number of cells
    
        if expected_doublet_scaled_rate == 0:
            expected_doublet_scaled_rate = 0.001 # double check
            stdev_doublet_rate = 0.0001
        else:
            stdev_doublet_rate = 0.02 #default
    
        # print('Dataset doubl. rate: {0:0.3f}. Corrected doub. rate: {1:0.3f}'.format(datasetDF.loc[studyID].expected_doublet_rate, expected_doublet_rate))
            
        # Run scrublet
        # Default mode; otherwise need to run after filtering genes and cells and selcting highly-variable genes.
        sc.external.pp.scrublet(adata_sub, expected_doublet_rate=expected_doublet_scaled_rate, stdev_doublet_rate=stdev_doublet_rate, verbose=False)
        
        scrublet_results_df_list.append(adata_sub.obs[['doublet_score','predicted_doublet']].round(3))
    
        # # Save results   
        # scrublet_results_df = scrublet_results_df.append(
        #     pd.DataFrame(list(zip(adata_sub.obs_names.to_list(),
        #                           [round(elem, 3) for elem in adata_sub.obs["doublet_score"]],
        #                           adata_sub.obs["predicted_doublet"])), 
        #                  columns=scrublet_results_df.columns))
        # scrublet_results_df = scrublet_results_df.reset_index(drop=True)    
        
    # Incorporate scrublet results into the filtered object
    # Set 'cell' as the index for 'scrublet_results_df'
    # scrublet_results_df.set_index('cell', inplace=True)
    
    scrublet_results_df = pd.concat(scrublet_results_df_list, axis=0, verify_integrity=True)
    
    # Saving the scrublet scores
    scrublet_results_df.to_pickle(here('00_data_processing/results/02_scrublet_score.pkl'))
else:
    scrublet_results_df = pd.read_pickle(here('00_data_processing/results/02_scrublet_score.pkl'))
    # Merge 'scrublet_results_df' with 'adata_sub.obs' based on the index

adata.obs = adata.obs.merge(scrublet_results_df, left_index=True, right_index=True, how='left')

# if we keep the columns with boolean type, it will raise errors during the saving at the end of the notebook
adata.obs.predicted_doublet = adata.obs.predicted_doublet.astype(int) 

  0%|                                                                                                                                 | 0/502 [00:00<?, ?it/s]

  0%|▏                                                                                                                      | 1/502 [00:16<2:20:58, 16.88s/it]

  0%|▍                                                                                                                      | 2/502 [00:33<2:18:08, 16.58s/it]

  1%|▋                                                                                                                      | 3/502 [00:49<2:16:27, 16.41s/it]

  1%|▉                                                                                                                      | 4/502 [01:06<2:19:19, 16.79s/it]

...

 99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎| 499/502 [2:09:02<00:37, 12.36s/it]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌| 500/502 [2:09:07<00:20, 10.01s/it]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 501/502 [2:09:08<00:07,  7.41s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 502/502 [2:09:12<00:00,  6.52s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 502/502 [2:09:12<00:00, 15.44s/it]

At this stage, we will not remove any putative doublet, but keep this information in the `adata.obs` object, to be considered in the downstream steps.

In [15]:
# Show top table of these values.
adata.obs.head()

,studyID,libraryID,sampleID,chemistry,technology,patientID,disease,timepoint_replicate,treatmentStatus,therapyResponse,sex,age,BMI,binned_age,diseaseStatus,smokingStatus,ethnicity,institute,diseaseGroup,n_genes_by_counts,log1p_n_genes_by_counts,total_counts,log1p_total_counts,pct_counts_in_top_20_genes,total_counts_mt,log1p_total_counts_mt,pct_counts_mt,total_counts_ribo,log1p_total_counts_ribo,pct_counts_ribo,total_counts_hb,log1p_total_counts_hb,pct_counts_hb,total_counts_plt,log1p_total_counts_plt,pct_counts_plt,doublet_score,predicted_doublet
cellID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
SCGT00_L051_I57.3P_T0_AAACCCAAGGTGAGAA,SCGT00,SCGT00_L051,SCGT00_I57.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I57,RA,0.0,ongoing,NR,female,47.0,25.71,41-50,na,never-smoker,na,na,IMIDs,406,6.008813,706,6.561031,26.062323,30,3.433987,4.249292,223,5.411646,31.586402,0,0.000000,0.000000,0,0.000000,0.000000,0.028,0
SCGT00_L051_I56.3P_T0_AAACCCAAGTCCGTCG,SCGT00,SCGT00_L051,SCGT00_I56.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I56,RA,0.0,ongoing,R,female,56.0,30.11,51-60,na,never-smoker,na,na,IMIDs,1091,6.995766,3804,8.244071,31.466877,145,4.983607,3.811777,1654,7.411556,43.480547,0,0.000000,0.000000,1,0.693147,0.026288,0.022,0
SCGT00_L051_I53.3P_T0_AAACCCAAGTGCACTT,SCGT00,SCGT00_L051,SCGT00_I53.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I53,RA,0.0,ongoing,R,female,69.0,25.00,61-70,na,never-smoker,na,na,IMIDs,2504,7.826044,9841,9.194414,23.757748,385,5.955837,3.912204,1945,7.573531,19.764252,3,1.386294,0.030485,11,2.484907,0.111777,0.019,0
SCGT00_L051_I52.3P_T0_AAACCCACAACTGTGT,SCGT00,SCGT00_L051,SCGT00_I52.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I52,RA,0.0,ongoing,NR,female,33.0,21.37,31-40,na,never-smoker,na,na,IMIDs,605,6.406880,1100,7.003974,34.909091,164,5.105945,14.909091,37,3.637586,3.363636,0,0.000000,0.000000,0,0.000000,0.000000,0.010,0
SCGT00_L051_I56.3P_T0_AAACCCACAAGAATGT,SCGT00,SCGT00_L051,SCGT00_I56.3P_T0,3_GEX_V3,3_GEX_V3_GenoHashed,SCGT00_I56,RA,0.0,ongoing,R,female,56.0,30.11,51-60,na,never-smoker,na,na,IMIDs,988,6.896694,4114,8.322394,28.415168,190,5.252273,4.618376,2296,7.739359,55.809431,0,0.000000,0.000000,1,0.693147,0.024307,0.018,0


## Assessing QC information

In [16]:
sc.pl.violin(
    adata, 
    [
     'total_counts', 
     'n_genes_by_counts', 
     'pct_counts_mt',
     'pct_counts_ribo',
     'pct_counts_hb',
     'pct_counts_plt'
    ],
    groupby="chemistry", 
    stripplot=False, rotation = 90,
)

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_totalCount_by_chemistry.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    
plt.show()

In [17]:
preFilterCellsIdx = adata.obs.studyID.isin(datasetDF.loc[datasetDF.expectedDoubletRate == '0'].index.tolist())
with plt.rc_context({'figure.figsize':(10,6)}):
    sc.pl.violin(adata[(adata.obs.chemistry == "3_GEX_V3") & (preFilterCellsIdx), :],
                 keys=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'], 
                 groupby='libraryID', 
                 stripplot=False,
                 rotation=90,
                 # title="3_GEX_V3"
                )
    if overwriteFigures:
        plt.savefig(here('00_data_processing/figures/02_QC_metrics_3GEXV3cleaned_totCount_byLibrary.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    plt.show()
    
    sc.pl.violin(adata[(adata.obs.chemistry == "3_GEX_V3") & (~preFilterCellsIdx), :],
                 keys=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'], 
                 groupby='libraryID', 
                 stripplot=False,
                 rotation=90,
                 # title="3_GEX_V3"
                )
    if overwriteFigures:
        plt.savefig(here('00_data_processing/figures/02_QC_metrics_3GEXV3notCleaned_totCount_byLibrary.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    plt.show()    
    
    
    
    
    sc.pl.violin(adata[(adata.obs.chemistry == "3_GEX_V2") & (preFilterCellsIdx), :], 
                 keys=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'], 
                 groupby='libraryID', 
                 stripplot=False,
                 rotation=90,
                 # title="3_GEX_V2"
                )
    if overwriteFigures:
        plt.savefig(here('00_data_processing/figures/02_QC_metrics_3GEXV2cleaned_totCount_byLibrary.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    plt.show()
    
    sc.pl.violin(adata[(adata.obs.chemistry == "3_GEX_V2") & (~preFilterCellsIdx), :], 
                 keys=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'], 
                 groupby='libraryID', 
                 stripplot=False,
                 rotation=90,
                 # title="3_GEX_V2"
                )
    if overwriteFigures:
        plt.savefig(here('00_data_processing/figures/02_QC_metrics_3GEXV2notCleaned_totCount_byLibrary.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    plt.show()    
    
    
    # 5_GEX 
    sc.pl.violin(adata[(adata.obs.chemistry == "5_GEX_V1") & (preFilterCellsIdx), :], 
                 keys=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'], 
                 groupby='libraryID', 
                 stripplot=False,
                 rotation=90,
                 # title="5_GEX_V1cleaned"
                )
    if overwriteFigures:
        plt.savefig(here('00_data_processing/figures/02_QC_metrics_5_GEX_V1cleaned_totCount_byLibrary.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    plt.show()
    
    sc.pl.violin(adata[(adata.obs.chemistry == "5_GEX_V1") & (~preFilterCellsIdx), :], 
                 keys=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'], 
                 groupby='libraryID', 
                 stripplot=False,
                 rotation=90,
                 # title="5_GEX_V1notCleaned"
                )
    if overwriteFigures:
        plt.savefig(here('00_data_processing/figures/02_QC_metrics_5_GEX_V1notCleaned_totCount_byLibrary.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    plt.show() 
    
        
    
    sc.pl.violin(adata[(adata.obs.chemistry == "5_GEX_V2") & (preFilterCellsIdx), :],
             keys=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'], 
             groupby='libraryID', 
             stripplot=False,
             rotation=90,
             # title="5_GEX_V2cleaned"
            )
    if overwriteFigures:
        plt.savefig(here('00_data_processing/figures/02_QC_metrics_5_GEX_V2cleaned_totCount_byLibrary.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    plt.show()
    
    sc.pl.violin(adata[(adata.obs.chemistry == "5_GEX_V2") & (~preFilterCellsIdx), :],
                 keys=['total_counts', 'n_genes_by_counts', 'pct_counts_mt'], 
                 groupby='libraryID', 
                 stripplot=False,
                 rotation=90,
                 # title="5_GEX_V2notCleaned"
                )
    if overwriteFigures:
        plt.savefig(here('00_data_processing/figures/02_QC_metrics_5_GEX_V2notCleaned_totCount_byLibrary.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
    plt.show()  

Next, we visualize the QC variables across libraries (considering the library' chemistry) and assess their overall quality based on the metrics to decide how to proceed with the QC step.

In [18]:
sns.displot(data=adata.obs, x="total_counts", col="chemistry", col_wrap=2, facet_kws={"sharex": False, "sharey": False})
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_metrics_displot_totalcounts_bychemistry.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


sns.displot(data=adata.obs, x="n_genes_by_counts", col="chemistry", col_wrap=2, facet_kws={"sharex": False, "sharey": False})
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_metrics_displot_genesbycounts_bychemistry.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


sns.displot(data=adata.obs, x="pct_counts_mt", col="chemistry", col_wrap=2, facet_kws={"sharex": False, "sharey": False})
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_metrics_displot_pctmt_bychemistry.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()

In [19]:
sc.pl.scatter(adata[adata.obs.chemistry == "3_GEX_V3", :],
              "total_counts", "n_genes_by_counts", color="pct_counts_mt", title="3_GEX_V3")
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_joinmetrics_3GEXV3_beforeFiltering.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


sc.pl.scatter(adata[adata.obs.chemistry == "3_GEX_V2", :],
              "total_counts", "n_genes_by_counts", color="pct_counts_mt", title="3_GEX_V2")
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_joinmetrics_3GEXV2_beforeFiltering.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


sc.pl.scatter(adata[adata.obs.chemistry == "5_GEX_V2", :],
              "total_counts", "n_genes_by_counts", color="pct_counts_mt", title="5_GEX_V2")
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_joinmetrics_5GEXV2_beforeFiltering.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


sc.pl.scatter(adata[adata.obs.chemistry == "5_GEX_V1", :],
              "total_counts", "n_genes_by_counts", color="pct_counts_mt", title="5_GEX_V1")
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_joinmetrics_5GEXV1_beforeFiltering.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()

After examining the previous results, we can see that the distribution of the three main QC covariates is really similar within the libraries generated using the same chemistry, and considering some datasets have already gone through a prior QC. For this reason, we will not perform a common QC for all of them, but split them by chemistry. 

It is important not to consider the QC covariates independently as this can lead to the misinterpretation of results, but to jointly explore how the three main metrics covary in order to set univariate QC thresholds. Additionally, it is also recommended to be quite permissive at this step and remove only very low quality cells, and later on, perform a second QC with more accurate and restrictive parameters (e.g. at the cell-lineage level).

In [20]:
for s in adata.obs.studyID.unique():
    
    sc.pl.scatter(adata[adata.obs.studyID == s, :],
                  "total_counts", "n_genes_by_counts", color="pct_counts_mt", title=s)
    
    if overwriteFigures:
        plt.savefig(here(f"00_data_processing/figures/02_QC_joinmetrics_{s}.pdf"), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
        
    plt.show()    

## Filter out low quality cells

According to the previous results, we have decided to apply different thresholds according to technology. Basically, cell barcodes with a low count depth, few detected genes, and a high fraction of mitochondrial counts could be indicative of lysed cells; therefore, we will proceed to filter them out.

In [21]:
# QC thresholds

cell_qc_thresholds = {
    "3_GEX_V3": {
        "min_libsize": 500,
        "max_libsize": 50000,        
        "min_libcomplex": 250,
        "max_libcomplex": 6000,
        "max_pctmt": 25
    },
    "3_GEX_V2": {
        "min_libsize": 500,
        "max_libsize": 25000,        
        "min_libcomplex": 250,
        "max_libcomplex": 4000,
        "max_pctmt": 20
    },
    "5_GEX_V2": {
        "min_libsize": 500,
        "max_libsize": 40000,        
        "min_libcomplex": 250,
        "max_libcomplex": 6000,
        "max_pctmt": 20
    },
    "5_GEX_V1": {
        "min_libsize": 500,
        "max_libsize": 50000,        
        "min_libcomplex": 250,
        "max_libcomplex": 6000,
        "max_pctmt": 20
    }
}

In [22]:
# Filter based on "chemistry" column
keepIDX = []

for chem_type, filter_params in cell_qc_thresholds.items():
    chem_filter = adata.obs["chemistry"] == chem_type
    chem_filter &= adata.obs["total_counts"] > filter_params["min_libsize"]
    chem_filter &= adata.obs["total_counts"] < filter_params["max_libsize"]
    chem_filter &= adata.obs["n_genes_by_counts"] > filter_params["min_libcomplex"]
    chem_filter &= adata.obs["n_genes_by_counts"] < filter_params["max_libcomplex"]
    chem_filter &= adata.obs["pct_counts_mt"] < filter_params["max_pctmt"]

    keepIDX += adata.obs.loc[chem_filter].index.tolist()

    # adata.append(adata[chem_filter])
    
adata = adata[adata.obs.index.isin(keepIDX)]
adata

View of AnnData object with n_obs × n_vars = 5771140 × 37169
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt', 'doublet_score', 'predicted_doublet'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'chemistry_colors'

**General statistics AFTER cell QC**

In [23]:
pd.set_option('display.max_rows', None)
qc_gex_df = qc_gex_projectlibrary(adata)
display(qc_gex_df)

pd.reset_option('display.max_rows')

,libraryID,chemistry,Number of cells,Median UMI counts,Median Genes,Median MT %,Median RB %,Median HB %,Median PLT %
0,"(COMBAT2022_L000,)",[5_GEX_V1],13853,3284,1149,1.52,29.85,0.00,0.06
1,"(COMBAT2022_L001,)",[5_GEX_V1],14320,3152,1112,1.61,30.18,0.00,0.06
2,"(COMBAT2022_L002,)",[5_GEX_V1],15258,2973,1063,1.61,30.49,0.00,0.06
3,"(COMBAT2022_L003,)",[5_GEX_V1],15561,3062,1081,1.61,30.72,0.00,0.06
4,"(COMBAT2022_L004,)",[5_GEX_V1],15567,3085,1094,1.62,30.65,0.00,0.06
5,"(COMBAT2022_L005,)",[5_GEX_V1],15455,2802,1010,1.52,30.88,0.00,0.06
6,"(COMBAT2022_L006,)",[5_GEX_V1],14916,2974,1062,1.51,30.25,0.00,0.06
7,"(COMBAT2022_L007,)",[5_GEX_V1],13397,3440,1130,2.20,30.34,0.00,0.07
8,"(COMBAT2022_L008,)",[5_GEX_V1],14375,3469,1130,2.12,31.00,0.00,0.07
9,"(COMBAT2022_L009,)",[5_GEX_V1],15059,3397,1121,2.20,30.97,0.00,0.07


In [24]:
sc.pl.scatter(adata[adata.obs.chemistry == "3_GEX_V3", :],
              "total_counts", "n_genes_by_counts", color="pct_counts_mt", title="3_GEX_V3")
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_joinmetrics_3GEXV3_afterFiltering.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


sc.pl.scatter(adata[adata.obs.chemistry == "3_GEX_V2", :],
              "total_counts", "n_genes_by_counts", color="pct_counts_mt", title="3_GEX_V2")
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_joinmetrics_3GEXV2_afterFiltering.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


sc.pl.scatter(adata[adata.obs.chemistry == "5_GEX_V2", :],
              "total_counts", "n_genes_by_counts", color="pct_counts_mt", title="5_GEX_V2")
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_joinmetrics_5GEXV2_afterFiltering.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


sc.pl.scatter(adata[adata.obs.chemistry == "5_GEX_V1", :],
              "total_counts", "n_genes_by_counts", color="pct_counts_mt", title="5_GEX_V1")
if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_joinmetrics_5GEXV1_afterFiltering.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()

In [25]:
# Now, the whole dataset
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mt")

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_joinmetrics_allDatasets_afterFiltering.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()

# Quality Control at (LIBRARY) & SAMPLE level

After exploring general library QC metrics, we decided to filter out the samples with very low number of cells, considering patient & timepoint, since some samples have been sequenced across multiple 10X libraries.

Also, after removing low quality cells, we ensure that there are now no low-quality library after removing low-quality cells, which would indicate low-quality library quality preparation and lead to a large batch-effect.

Lastly, we are removing cells that were captured in specific sample/library when the total number of cells belonging to this patient is smaller than 50 cells.

In [26]:
# QC thresholds
min_cells_by_sample = 500
min_meangenes_by_sample = 500
min_cells_by_samplelibrarypatient = 50

## Filter out low quality libraries

In [27]:
# Removing libraries with very low total number of cells (after cellQC)
min_cells_by_lib = 250

# Calculate the value counts for each 'libraryID'
library_cells = adata.obs["libraryID"].value_counts()

# Get the library names to remove
libraries_to_remove = library_cells[library_cells < min_cells_by_lib].index

if len(libraries_to_remove) > 0:
    print('Libraries removed:')
    print(libraries_to_remove.astype(str))

    # Filter the 'adata' object
    adata = adata[~adata.obs["libraryID"].isin(libraries_to_remove.astype(str))]
else:
    print('No library is removed')

No library is removed


## Filter out low quality samples

In [28]:
# Removing samples (patient & timepoint) with very low total number of cells

# Calculate the value counts for each 'sampleID'
sample_cells = adata.obs["sampleID"].value_counts()

# Get the sample names to remove
samples_to_remove = sample_cells[sample_cells < min_cells_by_sample].index

if len(samples_to_remove) > 0:
    print('Samples removed:')
    print(samples_to_remove.astype(str))

    # Filter the 'adata' object
    adata = adata[~adata.obs["sampleID"].isin(samples_to_remove.astype(str))]
else:
    print('No sample is removed')

Samples removed:
Index(['Reyes2020_C2P05_T0', 'Reyes2020_C2P21_T0', 'COMBAT2022_U00504_T0',
       'COMBAT2022_G05092_T0', 'COMBAT2022_S00030_T0'],
      dtype='object', name='sampleID')


In [29]:
# Removing samples with very low median number of total genes

# Calculate the median total counts for each 'sampleID'
sample_median_counts = adata.obs.groupby("sampleID")["n_genes_by_counts"].median()

# Get the sample names with median "total_counts" fewer than 500
libraries_to_remove = sample_median_counts[sample_median_counts < min_meangenes_by_sample].index

if len(libraries_to_remove) > 0:
    print('Samples removed:')
    print(libraries_to_remove.astype(str))

    # Filter the 'adata' object
    adata = adata[~adata.obs["sampleID"].isin(libraries_to_remove.astype(str))]
else:
    print('No sample is removed')

Samples removed:
Index(['Perez2022_HC019_T0', 'Perez2022_HC532_T0', 'Perez2022_HC574_T0'], dtype='object', name='sampleID')


## Filtering cells from low quality samples / library by patient

In [30]:
samplelibpatients_cells = adata.obs[["sampleID", "libraryID", "patientID"]].value_counts().sort_values(ascending=True)
samplelibpatients_cells.head()

sampleID               libraryID        patientID         
Perez2022_IGTB1906_T0  Perez2022_L033   Perez2022_IGTB1906     1
                       Perez2022_L035   Perez2022_IGTB1906     1
Reyes2020_C2P02_T0     Reyes2020_L028   Reyes2020_C2P02        7
Reyes2020_C2P01_T0     Reyes2020_L028   Reyes2020_C2P01        7
COMBAT2022_U00601_T0   COMBAT2022_L062  COMBAT2022_U00601     68
Name: count, dtype: int64

In [31]:
# Removing CELLS (sample & library & patient) with very low total number of cells

# Calculate the value counts
samplelibpatients_cells = adata.obs[["sampleID", "libraryID", "patientID"]].value_counts().sort_values(ascending=True)
samplelibpatients_cells.head()

# Get the sample names to remove
cells_to_remove_df = samplelibpatients_cells[samplelibpatients_cells < min_cells_by_samplelibrarypatient]
cells_to_remove_df = pd.DataFrame(cells_to_remove_df.reset_index())
cells_to_remove_df["libraryID"] = cells_to_remove_df["libraryID"].astype(str)
cells_to_remove_df["sampleID"] = cells_to_remove_df["sampleID"].astype(str)
cells_to_remove_df['startwith_cellID'] = cells_to_remove_df["libraryID"] + '_' + cells_to_remove_df["sampleID"].str.split("_", expand=True)[1] + '_' + cells_to_remove_df["sampleID"].str.split("_", expand=True)[2]

if len(cells_to_remove_df) > 0:

    cellID_to_remove = [cell_id for cell_id in adata.obs_names if any(cell_id.startswith(startwith) 
                             for startwith in cells_to_remove_df["startwith_cellID"])]
    print('Cells from samples/library/patient removed:')
    print(len(cellID_to_remove))
    print(cellID_to_remove)

    # Filter the 'adata' object
    adata = adata[~adata.obs_names.isin(cellID_to_remove)]
else:
    print('No cells from samples/library/patient are removed.')

Cells from samples/library/patient removed:
16
['Reyes2020_L028_C2P01_T0_ACATCAGGTCCATCCT', 'Reyes2020_L028_C2P02_T0_AGTGGGACACCGGAAA', 'Reyes2020_L028_C2P01_T0_CGATCGGTCTCAAACG', 'Reyes2020_L028_C2P01_T0_CTGATAGCAGTGGAGT', 'Reyes2020_L028_C2P01_T0_CTTACCGCAGGTCTCG', 'Reyes2020_L028_C2P02_T0_GAAACTCGTGTAAGTA', 'Reyes2020_L028_C2P02_T0_GGACAGAAGAGGACGG', 'Reyes2020_L028_C2P01_T0_GGAGCAACATCTCCCA', 'Reyes2020_L028_C2P01_T0_GGGCATCTCGTACCGG', 'Reyes2020_L028_C2P02_T0_GTGTGCGCACATTTCT', 'Reyes2020_L028_C2P02_T0_TGAGAGGCAGACGCAA', 'Reyes2020_L028_C2P02_T0_TGGACGCGTCGCCATG', 'Reyes2020_L028_C2P02_T0_TTGGAACCAAGTCTGT', 'Reyes2020_L028_C2P01_T0_TTTACTGGTGTTCTTT', 'Perez2022_L033_IGTB1906_T0_ATAAGAGAGGTGATAT', 'Perez2022_L035_IGTB1906_T0_AAAGCAACAGGTCTCG']


In [32]:
adata

View of AnnData object with n_obs × n_vars = 5768062 × 37169
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt', 'doublet_score', 'predicted_doublet'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'chemistry_colors'

In [33]:
len(adata.obs["sampleID"].unique())

961

# Quality control at GENES level

After ensuring the integrity of cells, we should also performed QC on the genes. snRNA-seq experiments usually include counts for a large number of genes (> 20000). However, some these genes have zero counts and can dramatically reduce the expression of a certain cell, and other genes are only expressed in a few cells and do not inform of the cellular heterogeneity, but might correspond to rare cell types lowly expressed. A possible approach to reduce the number of genes is to filter out gens expressed in few cells than a specific threshold value based on the distribution of our data.

After examining the previous results, we can see that the distribution of the three main QC covariates is really similar within the libraries generated using the same chemistry, and considering some datasets have already gone through a prior QC. For this reason, we will not perform a common QC for all of them, but split them by chemistry. 

In [35]:
# QC thresholds
min_cells_by_gene = 10
min_samples_by_gene = 5

**Distribution of # cells expressing each gene**

In [36]:
# Number of cells expressing each gene, excluding 0 counts
n_cells = np.sum(adata.X > 0, axis=0)
n_cells_df = np.ravel(n_cells)

In [37]:
# Plot the histogram
plt.figure(figsize=(12, 6))
plt.hist(n_cells_df, bins=100, alpha=0.75)
#plt.xscale('log',base=10) 
plt.xlabel('Number of Expressing Cells')
plt.ylabel('Gene Frequency')

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_genesExpressionFrequencyCells.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()

**Distribution of # sample expressing genes in how many cells**

In [38]:
n_cellsSample_df_list = []#pd.DataFrame()

# Binarize the sparse matrix DataFrame

pbar = tqdm(total = adata.obs["sampleID"].unique().shape[0])
# Looping for samples
for i, sampleID in enumerate(adata.obs["sampleID"].unique()):
    # Subsetting adata object by sampleID
    adata_sub = adata[adata.obs["sampleID"] == sampleID]
    # Binarize the sparse matrix 
    binarized_matrix = (adata_sub.X > 0)
    # Number of cells expressing each gene, excluding 0 counts
    n_cellsSample = np.sum(binarized_matrix, axis=0)
    # Storing gene counts
    n_cellsSample_df_list.append(pd.DataFrame(n_cellsSample))
    # if i == 0:
    #     n_cellsSample_df = pd.DataFrame(n_cellsSample)
    # else:        
    #     n_cellsSample_df = pd.concat([n_cellsSample_df, pd.DataFrame(n_cellsSample)], ignore_index=True)
    pbar.update(1)
n_cellsSample_df = pd.concat(n_cellsSample_df_list, ignore_index=True)
n_cellsSample_df.head()

  0%|                                                                                                                                 | 0/961 [00:00<?, ?it/s]

  0%|▏                                                                                                                        | 1/961 [00:00<07:50,  2.04it/s]

  0%|▎                                                                                                                        | 2/961 [00:00<07:33,  2.11it/s]

  0%|▍                                                                                                                        | 3/961 [00:01<07:09,  2.23it/s]

  0%|▌                                                                                                                        | 4/961 [00:02<09:23,  1.70it/s]

...

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▋| 958/961 [08:50<00:03,  1.07s/it]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▊| 959/961 [08:51<00:01,  1.07it/s]

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉| 960/961 [08:52<00:00,  1.13it/s]

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 961/961 [08:53<00:00,  1.07s/it]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,...,37019,37020,37021,37022,37023,37024,37025,37026,37027,37028,37029,37030,37031,37032,37033,37034,37035,37036,37037,37038,37039,37040,37041,37042,37043,37044,37045,37046,37047,37048,37049,37050,37051,37052,37053,37054,37055,37056,37057,37058,37059,37060,37061,37062,37063,37064,37065,37066,37067,37068,37069,37070,37071,37072,37073,37074,37075,37076,37077,37078,37079,37080,37081,37082,37083,37084,37085,37086,37087,37088,37089,37090,37091,37092,37093,37094,37095,37096,37097,37098,37099,37100,37101,37102,37103,37104,37105,37106,37107,37108,37109,37110,37111,37112,37113,37114,37115,37116,37117,37118,37119,37120,37121,37122,37123,37124,37125,37126,37127,37128,37129,37130,37131,37132,37133,37134,37135,37136,37137,37138,37139,37140,37141,37142,37143,37144,37145,37146,37147,37148,37149,37150,37151,37152,37153,37154,37155,37156,37157,37158,37159,37160,37161,37162,37163,37164,37165,37166,37167,37168
0,5,0,408,138,32,1046,37,212,204,156,47,300,347,163,5,0,307,590,349,39,303,844,2230,25,0,4,1,657,865,160,373,1090,3,0,46,12,62,239,1872,28,160,1369,15,0,356,94,245,323,395,291,139,278,1079,92,579,166,1,30,873,139,4,12,3,0,276,33,350,7,1,165,108,3,290,918,2239,763,0,1606,118,11,0,280,503,53,5,277,252,261,17,156,1028,1040,256,22,2,144,406,131,48,2997,348,11,470,301,1726,639,1266,114,26,766,782,142,0,357,574,5,130,2,0,0,260,1,0,2,1052,0,153,1,82,0,169,0,65,381,9,570,2,284,258,19,545,0,208,0,340,1039,17,1764,8,230,...,0,0,8,0,1,0,0,0,0,0,0,0,0,0,0,1,0,3,0,0,4,0,9,0,0,57,0,0,4,0,5,0,0,0,0,10,0,7,0,1,0,2,11,25,0,0,0,8,0,3,4,1,0,5,198,0,1,0,0,0,5,0,0,0,0,0,0,0,0,0,21,2,1,8,3,140,3,0,0,1,0,0,0,0,33,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,103,0,6,37,1,0,0,0,0,0,0,0,12,1,1,0,0,0,6,141,0,0,1,0,0,12,0,0,18,0,0,140,0,28,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,5,0,543,111,31,1199,16,277,180,148,33,218,329,204,19,0,276,875,357,30,285,915,2074,15,1,2,1,394,752,163,165,1118,6,0,58,16,77,250,1762,49,232,1369,10,1,404,64,202,366,422,182,160,288,911,160,537,161,1,37,919,155,16,8,6,0,345,29,365,7,2,284,106,4,290,803,2350,818,0,1383,69,8,1,260,512,67,2,289,224,413,26,206,974,1069,157,31,0,116,332,117,17,3273,489,5,381,226,1397,711,1030,92,18,774,955,141,0,424,558,11,115,5,0,0,272,0,0,2,886,0,182,0,73,0,78,1,56,552,11,764,3,248,404,25,479,3,179,0,358,945,18,1384,4,176,...,0,0,4,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,2,0,13,0,0,55,0,0,1,0,5,1,0,0,0,11,0,9,0,1,0,0,6,12,0,0,0,2,0,2,8,0,0,4,258,0,1,0,0,0,7,0,0,0,0,0,0,0,1,0,19,0,0,5,3,85,5,0,0,0,0,0,0,1,19,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,63,0,5,27,0,0,0,0,0,0,0,0,5,1,0,0,0,0,4,128,0,0,0,0,0,0,0,0,20,0,0,189,0,22,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,7,0,764,194,16,840,25,177,166,97,29,265,242,217,82,0,194,642,339,21,234,727,2129,34,0,2,4,271,829,188,264,774,3,0,42,5,53,179,1762,123,260,1338,15,0,423,63,213,284,196,141,77,218,767,160,462,139,0,32,1123,177,7,7,6,1,313,74,511,4,2,274,73,4,426,177,1929,637,0,1067,69,3,0,237,285,32,2,344,348,418,6,94,768,1233,380,17,0,90,250,116,8,3245,762,3,346,216,790,757,656,59,17,837,774,76,0,467,333,3,117,1,0,0,478,0,0,4,648,0,168,2,138,2,134,0,69,850,14,758,2,296,225,16,489,9,271,0,166,1174,5,1109,3,161,...,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,0,1,22,1,0,0,0,1,0,0,0,0,11,0,3,0,0,0,0,11,37,0,0,0,1,0,0,4,0,0,2,140,0,2,0,0,0,11,0,1,0,0,0,1,0,0,0,20,0,0,12,2,139,7,0,0,0,0,0,1,0,14,1,0,0,2,1,0,0,0,1,0,0,0,0,0,0,0,0,101,0,4,99,0,0,0,0,0,0,0,0,1,4,0,0,0,0,5,98,0,0,0,0,0,0,0,0,19,0,0,149,0,29,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19,0,1195,307,56,1922,73,436,406,168,56,316,542,829,61,0,386,1623,565,41,510,1387,4388,24,1,10,5,691,1297

In [39]:
n_cellsSample_df.shape

(961, 37169)

In [40]:
# Number of samples expressing each gene in at least some cells, excluding 0 counts
n_sample_df = np.sum(n_cellsSample_df > min_cells_by_gene, axis=0)

# Plot the histogram
plt.figure(figsize=(12, 6))
plt.hist(n_sample_df, bins=100, alpha=0.75)
#plt.xscale('log',base=10) 
plt.xlabel('Number of Expressing Samples')
plt.ylabel('Gene Frequency')

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_genesExpressionFrequencySample.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()

We also want to see how the expression change among different studies. For doing so we computed the average expression per study for each genes. Than, we binarize the result, using a threshold = 0. Thus, genes are considered expressed if they are detected in at least 1 cell for a given study.

Considering all the genes we get the following heatmap 

In [41]:
qc_gene_expression_per_study(adata, save_name=here('00_data_processing/figures/02_QC_genenames_all.pdf'), 
                             fig_size = (20,12))

Then, we consider only genes with an official HUGO symbol

In [42]:
qc_gene_expression_per_study(adata, 
                             gene_list=adata.var.index[adata.var.HUGO_status == 'official'].to_list(),
                             save_name=here('00_data_processing/figures/02_QC_genenames_HUGOofficial.pdf'),
                             fig_size = (20,12))

In [43]:
qc_gene_expression_per_study(adata, 
                             gene_list=adata.var.index[adata.var.HUGO_status == 'non-official'].to_list(),
                             save_name=here('00_data_processing/figures/02_QC_genenames_HUGO_non-official.pdf'),
                             fig_size = (20,12))

As you can see in both cases, many genes are not expressed in any study (most of them does not have a official hugo symbol). Therefore, we find reasonable to remove genes that are expressed in less than 10 cells.

## Filter out genes expressed in few cells

After ensuring the integrity of cells, we should also performed QC on the genes. snRNA-seq experiments usually include counts for a large number of genes (20-30K). However, some these genes have zero counts and can dramatically reduce the expression of a certain cell, and other genes are only expressed in a few cells, specially in such large studies with data coming from multiple sources.

A possible approach to reduce the number of genes is to filter out gens expressed in fewer cells or fewer samples than a specific threshold value. According to Luecken MD et al. "A guideline to setting this threshold is to use the minimum cell cluster size that is of interest and leaving some leeway for dropout effects. The choice of threshold should scale with the number of cells in the dataset and the intended downstream analysis."

In [44]:
# QC thresholds
min_cells_by_gene = 20
min_samples_by_gene = 5

In [45]:
# Check each GEN for more than 5 SAMPLES with values larger than 20
genes_to_keep = (n_cellsSample_df > min_cells_by_gene).sum(axis=0) > min_samples_by_gene
# Convert the result to a list of True or False
genes_to_keep = genes_to_keep.tolist()

num_genes_to_keep = np.count_nonzero(genes_to_keep)
print(num_genes_to_keep)

22838


In [46]:
# Convert the boolean list to a numpy array
gene_mask_array = np.array(genes_to_keep)

# Subset the .X dataframe based on the boolean list
adata = adata[:, gene_mask_array]
adata

View of AnnData object with n_obs × n_vars = 5768062 × 22838
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt', 'doublet_score', 'predicted_doublet'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'chemistry_colors'

After the filtering we can generate again the two heatmap to shows the expression profiles among studies.
So, considering all the remaining genes, we get the following plot.

In [47]:
qc_gene_expression_per_study(adata, 
                             save_name=here('00_data_processing/figures/02_QC_genenames_all_postFilter.pdf'), 
                             fig_size = (20,12))

Considering only the *official* ones, we get the next plot

In [48]:
qc_gene_expression_per_study(adata, 
                             gene_list=adata.var.index[adata.var.HUGO_status == 'official'].to_list(),
                            save_name=here('00_data_processing/figures/02_QC_genenames_HUGOofficial_postFilter.pdf'), 
                             fig_size = (20,12))

In [49]:
'Number of genes with official HUGO symbol {}'.format(len(adata.var.index[adata.var.HUGO_status == 'official'].to_list()))

'Number of genes with official HUGO symbol 18772'

The next plot shows the profile of the genes without an official HUGO symbol

In [50]:
qc_gene_expression_per_study(adata,gene_list=adata.var.index[adata.var.HUGO_status == 'non-official'].to_list(),
                             save_name=here('00_data_processing/figures/02_QC_genenames_HUGO_non-official_postFilter.pdf'), 
                             fig_size = (20,12))

As you can see, most of those genes are expressed only in a subset of studies. Probably is due to the version of the annotation file used during the alignment. However, there are also some that are expressed in all the studies. Therefore, to not bias the results with an arbitrary decision, we are not removing any of them. Instead, we will propose a strategy to select the High Variable Genes (HVG) in the next notebook, that takes this aspect into account. 

# Save results

In [51]:
# Save Anndata object (.h5ad file) 
adata.write(here("00_data_processing/results/02_INFLAMMATION_core_afterQC.h5ad"), compression="gzip")
del adata

# Split CORE data downstream analysis

As anticipated before, we aim to split the **CORE DATASET** into two major datasets (named, MAIN and VALIDATION) in a balanced manner, considering some relevant covariates, such as *studyID*, *chemistry* and *disease* to ensure an homogeneous sampling. 

* The **main** dataset will be employed to characterize the inflammation landscape of circulating immune cells and test the classifier with a 5-fold cross-validation strategy. 

* The **validation** dataset will be kept out to assess the performances of the patient classifier.

In [52]:
infile_path = here("00_data_processing/results/02_INFLAMMATION_core_afterQC.h5ad")
adata_core = sc.read_h5ad(infile_path)
adata_core

AnnData object with n_obs × n_vars = 5768062 × 22838
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt', 'doublet_score', 'predicted_doublet'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'chemistry_colors'

In [53]:
adata_core.obs[["studyID", "chemistry", "disease", "diseaseStatus", "sex", "binned_age"]].groupby("studyID").nunique()

,chemistry,disease,diseaseStatus,sex,binned_age
studyID,,,,,
COMBAT2022,1,4,8,2,7
Cillo2020,1,2,10,2,6
Liu2021,1,1,7,1,4
Perez2022,1,2,3,2,7
Ren2021,2,2,5,2,7
Reyes2020,1,2,4,2,6
SCGT00,1,7,2,2,7
SCGT01,1,1,1,1,2
SCGT02,1,3,7,2,4


In [54]:
grouped = adata_core.obs.groupby(["studyID", "chemistry", "disease"], observed=True)
results = grouped["patientID"].nunique().reset_index().rename(columns={"patientID": "patient_count"})
results = results[results["patient_count"] < 10].reset_index()
results

,index,studyID,chemistry,disease,patient_count
0,5,Cillo2020,3_GEX_V2,healthy,6
1,11,Ren2021,5_GEX_V2,healthy,5
2,20,SCGT00,3_GEX_V3,healthy,8
3,21,SCGT01,3_GEX_V3,healthy,7
4,24,SCGT02,3_GEX_V3,healthy,8
5,25,SCGT03,5_GEX_V2,BRCA,6
6,26,SCGT04,3_GEX_V3,cirrhosis,8
7,28,Schafflick2020,3_GEX_V2,MS,5
8,29,Schafflick2020,3_GEX_V2,healthy,5
9,31,Wang2020,3_GEX_V2,HIV,6


In [55]:
# studyID, chemistry, disease; not considering sex, binned_age, diseaseStatus (opt),
# compute the number of patients in each categories -> decide if there are enough patients to sample them
validation_patientIDs = []

adata_obs_df = adata_core.obs[["studyID", "patientID", "chemistry", "disease", "diseaseStatus", "sex", "binned_age"]].drop_duplicates()
df_validation = balanced_sample(adata_obs_df, cols = ["studyID", "chemistry", "disease"], frac = 0.2, random_state = 42)
validation_patientIDs = df_validation["patientID"].unique().tolist()
print(len(validation_patientIDs))

191


In [56]:
validation_grouped = adata_core.obs[adata_core.obs.patientID.isin(validation_patientIDs)].groupby(["studyID", "chemistry", "disease"], observed=True)
validation_results = validation_grouped["patientID"].nunique().reset_index().rename(columns={"patientID": "patient_count"})
validation_results.sort_values(by="patient_count", ascending=False)

,studyID,chemistry,disease,patient_count
30,Terekhova2023,5_GEX_V2,healthy,33
7,Perez2022,3_GEX_V2,SLE,32
8,Perez2022,3_GEX_V2,healthy,18
0,COMBAT2022,5_GEX_V1,COVID,15
17,SCGT00,3_GEX_V3,RA,10
10,Ren2021,5_GEX_V2,COVID,9
16,SCGT00,3_GEX_V3,PSA,6
15,SCGT00,3_GEX_V3,PS,6
14,SCGT00,3_GEX_V3,CD,5
4,Cillo2020,3_GEX_V2,HNSCC,5


After observing the number of sampled patients from each category (studyID, chemistry and disease) for the **validation** dataset, and considering we aim to have statistical power for the downstream analysis to assess the classifier performance, we will only keep sampled patients belonging to groups (studyID, chemistry and disease) for which we have an equal or larger than 5 patients.

In [57]:
# Keeping validation_patientIDS only if >=5 patients per group stratification (studyID, chemistry and disease)
min_patientsStratified_validation = 5

validation_results_filtered = validation_results[validation_results["patient_count"] >= min_patientsStratified_validation].reset_index()
validation_results_filtered

,index,studyID,chemistry,disease,patient_count
0,0,COMBAT2022,5_GEX_V1,COVID,15
1,3,COMBAT2022,5_GEX_V1,sepsis,5
2,4,Cillo2020,3_GEX_V2,HNSCC,5
3,7,Perez2022,3_GEX_V2,SLE,32
4,8,Perez2022,3_GEX_V2,healthy,18
5,10,Ren2021,5_GEX_V2,COVID,9
6,14,SCGT00,3_GEX_V3,CD,5
7,15,SCGT00,3_GEX_V3,PS,6
8,16,SCGT00,3_GEX_V3,PSA,6
9,17,SCGT00,3_GEX_V3,RA,10


In [58]:
# Merge validation_results_sorted with the original DataFrame to retrieve patient IDs
validation_results_merged = pd.merge(validation_results_filtered, adata_core.obs[['studyID', 'chemistry', 'disease', 'patientID']], 
                                     on=['studyID', 'chemistry', 'disease'], how='inner')
# Filter by validation_patientIDs list
validation_results_merged = validation_results_merged[validation_results_merged['patientID'].isin(validation_patientIDs)]
# Filter the merged DataFrame based on the condition
validation_results_merged = validation_results_merged.groupby(["studyID", "chemistry", "disease"]).filter(
    lambda x: x['patient_count'].sum() >= 5)

# Extract patient IDs from the filtered DataFrame
validation_patientIDs_filtered = validation_results_merged['patientID'].unique().tolist()
print(len(validation_patientIDs_filtered))

144


**Splitting CORE dataset**

In [59]:
adata_validation = adata_core[adata_core.obs.patientID.isin(validation_patientIDs_filtered)]
adata_validation

View of AnnData object with n_obs × n_vars = 849922 × 22838
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt', 'doublet_score', 'predicted_doublet'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'chemistry_colors'

In [60]:
adata_main = adata_core[~adata_core.obs.patientID.isin(validation_patientIDs_filtered)]
adata_main

View of AnnData object with n_obs × n_vars = 4918140 × 22838
    obs: 'studyID', 'libraryID', 'sampleID', 'chemistry', 'technology', 'patientID', 'disease', 'timepoint_replicate', 'treatmentStatus', 'therapyResponse', 'sex', 'age', 'BMI', 'binned_age', 'diseaseStatus', 'smokingStatus', 'ethnicity', 'institute', 'diseaseGroup', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'total_counts_plt', 'log1p_total_counts_plt', 'pct_counts_plt', 'doublet_score', 'predicted_doublet'
    var: 'hgnc_id', 'symbol', 'locus_group', 'HUGO_status', 'mt', 'ribo', 'hb', 'plt', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'chemistry_colors'

In [61]:
# Save Anndata object (.h5ad file) 
adata_main.write(here("00_data_processing/results/02_INFLAMMATION_main_afterQC.h5ad"), compression="gzip")
adata_validation.write(here("00_data_processing/results/02_INFLAMMATION_validation_afterQC.h5ad"), compression="gzip")

**Saving patient metadata**

In [3]:
main_sample_metadata = adata_main.obs[['studyID','sampleID','chemistry','technology','patientID','disease','timepoint_replicate','treatmentStatus',
                    'therapyResponse','sex','age','BMI','binned_age','diseaseStatus','smokingStatus','ethnicity','institute','diseaseGroup']].drop_duplicates()
assert main_sample_metadata.shape[0] == main_sample_metadata.sampleID.unique().shape[0]

In [4]:
validation_sample_metadata = adata_validation.obs[['studyID','sampleID','chemistry','technology','patientID','disease','timepoint_replicate','treatmentStatus',
                    'therapyResponse','sex','age','BMI','binned_age','diseaseStatus','smokingStatus','ethnicity','institute','diseaseGroup']].drop_duplicates()
assert validation_sample_metadata.shape[0] == validation_sample_metadata.sampleID.unique().shape[0]

In [5]:
main_sample_metadata.to_pickle(here("00_data_processing/results/01_INFLAMMATION_main_sampleMetadata.pkl"))
validation_sample_metadata.to_pickle(here("00_data_processing/results/01_INFLAMMATION_validation_sampleMetadata.pkl"))


**Asessing sex and age distribution**

In [62]:
adata_obs_core = adata_core.obs[["patientID", "studyID", "chemistry", "binned_age", "sex", "disease"]].drop_duplicates()

plt.figure(figsize=(12, 20))
catplot = sns.catplot(x="disease", hue="binned_age", col="sex", kind="count", data=adata_obs_core,  height=5, aspect=1.5)
for ax in catplot.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_COREdata_SexAgeDistribution.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


plt.figure(figsize=(12, 20))
catplot = sns.catplot(x="disease", hue="chemistry", col="sex", kind="count", data=adata_obs_core,  height=5, aspect=1.5)
for ax in catplot.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_COREdata_SexChemistryDistribution.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()

In [63]:
adata_obs_main = adata_main.obs[["patientID", "studyID", "chemistry", "binned_age", "sex", "disease"]].drop_duplicates()

plt.figure(figsize=(12, 20))
catplot = sns.catplot(x="disease", hue="binned_age", col="sex", kind="count", data=adata_obs_main,  height=5, aspect=1.5)
for ax in catplot.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_MAINdata_SexAgeDistribution.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


plt.figure(figsize=(12, 20))
catplot = sns.catplot(x="disease", hue="chemistry", col="sex", kind="count", data=adata_obs_main,  height=5, aspect=1.5)
for ax in catplot.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_MAINdata_SexChemistryDistribution.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()

In [64]:
adata_obs_validation = adata_validation.obs[["patientID", "studyID", "chemistry", "binned_age", "sex", "disease"]].drop_duplicates()

plt.figure(figsize=(12, 20))
catplot = sns.catplot(x="disease", hue="binned_age", col="sex", kind="count", data=adata_obs_validation,  height=5, aspect=1.5)
for ax in catplot.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_VALIDATIONdata_SexAgeDistribution.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()


plt.figure(figsize=(12, 20))
catplot = sns.catplot(x="disease", hue="chemistry", col="sex", kind="count", data=adata_obs_validation,  height=5, aspect=1.5)
for ax in catplot.axes.flat:
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90)

if overwriteFigures:
    plt.savefig(here('00_data_processing/figures/02_QC_VALIDATIONdata_SexChemistryDistribution.pdf'), bbox_inches='tight', pad_inches=0, dpi=dpi_fig_save)
plt.show()

In [65]:
session_info.show()